In [44]:
import pathlib
import pandas as pd

pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 20)
pd.set_option("display.width", 100)

In [69]:
df = pd.read_table("../../../snakemake_workflow/samplesheets/final_samplesheet.tsv")

In [70]:
# get list of sample_uids where there is gene expression
gex_libs = df[df.lib_type == 'gex'].sample_uid.values

In [71]:
table = df[df.lib_type == 'vdj']

In [74]:
pooled_gex = table[(~table.sample_uid.isin(gex_libs))]

In [80]:
relationships = pd.read_table("../../../snakemake_workflow/samplesheets/sample_relationships.tsv")

In [81]:
relationships

,sample_uid,sample_uid_vdj,sample_uid_gex,encapsulation_sibling_sample,is_gex_sample,cell_suspension,donor_subanatomical,donor_tissue,donor
0,TBd1_frozen_BM_HTa,TBd1_frozen_BM_HTa,TBd1_frozen_BM_HTa,TBd1_frozen_BM_HTb,True,TBd1_frozen_BM,TBd1_BM,TBd1_BM,TBd1
1,TBd1_frozen_BM_HTb,TBd1_frozen_BM_HTb,TBd1_frozen_BM_HTb,TBd1_frozen_BM_HTa,True,TBd1_frozen_BM,TBd1_BM,TBd1_BM,TBd1
2,TBd1_frozen_BMover_HTa,TBd1_frozen_BMover_HTa,NaN,TBd1_frozen_BMover_HTb,False,TBd1_frozen_BM,TBd1_BM,TBd1_BM,TBd1
3,TBd1_frozen_BMover_HTb,TBd1_frozen_BMover_HTb,NaN,TBd1_frozen_BMover_HTa,False,TBd1_frozen_BM,TBd1_BM,TBd1_BM,TBd1
4,TBd1_frozen_MELN_HTa,TBd1_frozen_MELN_HTa,TBd1_frozen_MELN_HTa,TBd1_frozen_MELN_HTb,True,TBd1_frozen_MELN,TBd1_MELN,TBd1_LN,TBd1
...,...,...,...,...,...,...,...,...,...
80,TBd6_frozen_SDLN1over_HTb,TBd6_frozen_SDLN1over_HTb,NaN,TBd6_frozen_SDLN1over_HTa,False,TBd6_frozen_SDLN1,TBd6_SDLN1,TBd6_LN,TBd6
81,TBd6_frozen_SDLN2_HTa,TBd6_frozen_SDLN2_HTa,TBd6_frozen_SDLN2_HT,TBd6_frozen_SDLN2_HTb,True,TBd6_frozen_SDLN2,TBd6_SDLN2,TBd6_LN,TBd6
82,TBd6_frozen_SDLN2_HTb,TBd6_frozen_SDLN2_HTb,TBd6_frozen_SDLN2_HT,TBd6_frozen_SDLN2_HTa,True,TBd6_frozen_SDLN2,TBd6_SDLN2,TBd6_LN,TBd6
83,TBd6_frozen_SDLN2over_HTa,TBd6_frozen_SDLN2over_HTa,NaN,TBd6_frozen_SDLN2over_HTb,False,TBd6_frozen_SDLN2,TBd6_SDLN2,TBd6_LN,TBd6


In [93]:
relationships.loc[:, 'negative_selection'] = (~relationships['donor_tissue'].str.contains("LN") & (~relationships['sample_uid'].str.contains("TBd5_frozen_PB")))

In [101]:
relationships = relationships[['cell_suspension', 'sample_uid', 'encapsulation_sibling_sample', 'is_gex_sample', 'negative_selection']]

In [102]:
relationships.columns = ["Cell Suspension", "Unique Sample ID", "Technical Sibling", "GEX", "B cell enriched"]

In [105]:
relationships.to_csv("~/latex_table.csv")

In [116]:
relationships['Technical Sibling'] = relationships['Technical Sibling'].fillna("")

/tmp/ipykernel_67992/1829705392.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relationships['Technical Sibling'] = relationships['Technical Sibling'].fillna("")


In [119]:
df = relationships

# Initialize the LaTeX table string
latex_table = "\\floatsetup[longtable]{LTcapwidth=table}\n"
latex_table += "\\begin{longtable}{lllp{1cm}p{1cm}p{1.3cm}}\n"
latex_table += "\\midrule\n"
latex_table += "Cell Suspension & Unique Sample ID & Technical Sibling & GEX & B{\\color{white}B}cell enriched \\\\\n"
latex_table += "\\midrule\n"
latex_table += "\\endhead\n"

# Loop through each row to populate the LaTeX table
for index, row in df.iterrows():
    gex_check = "\\checkmark" if row['GEX'] else ""
    b_cell_check = "\\checkmark" if row['B cell enriched'] else ""
    latex_table += f"\\verb|{row['Cell Suspension']}| & \\verb|{row['Unique Sample ID']}| & \\verb|{row['Technical Sibling']}| & {gex_check} & {b_cell_check} \\\\\n"


In [120]:
print(latex_table)

\floatsetup[longtable]{LTcapwidth=table}
\begin{longtable}{lllp{1cm}p{1cm}p{1.3cm}}
\midrule
Cell Suspension & Unique Sample ID & Technical Sibling & GEX & B{\color{white}B}cell enriched \\
\midrule
\endhead
\verb|TBd1_frozen_BM| & \verb|TBd1_frozen_BM_HTa| & \verb|TBd1_frozen_BM_HTb| & \checkmark & \checkmark \\
\verb|TBd1_frozen_BM| & \verb|TBd1_frozen_BM_HTb| & \verb|TBd1_frozen_BM_HTa| & \checkmark & \checkmark \\
\verb|TBd1_frozen_BM| & \verb|TBd1_frozen_BMover_HTa| & \verb|TBd1_frozen_BMover_HTb| &  & \checkmark \\
\verb|TBd1_frozen_BM| & \verb|TBd1_frozen_BMover_HTb| & \verb|TBd1_frozen_BMover_HTa| &  & \checkmark \\
\verb|TBd1_frozen_MELN| & \verb|TBd1_frozen_MELN_HTa| & \verb|TBd1_frozen_MELN_HTb| & \checkmark &  \\
\verb|TBd1_frozen_MELN| & \verb|TBd1_frozen_MELN_HTb| & \verb|TBd1_frozen_MELN_HTa| & \checkmark &  \\
\verb|TBd1_frozen_MELN| & \verb|TBd1_frozen_MELNover_HTa| & \verb|TBd1_frozen_MELNover_HTb| &  &  \\
\verb|TBd1_frozen_MELN| & \verb|TBd1_frozen_MELNover_HTb| & \